In [2]:
import pandas as pd
import numpy as np

In [4]:
col_names = ['unit', 'cycle', 'op_set_1', 'op_set_2', 'op_set_3',
             'sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6',
             'sensor7', 'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 
             'sensor13', 'sensor14', 'sensor15', 'sensor16', 'sensor17',
             'sensor18', 'sensor19', 'sensor20', 'sensor21']


data = pd.read_csv('train_FD001.txt',
                   delimiter=' ',
                   names=col_names,
                   index_col=False,
                   skipinitialspace=True
                   )

data.set_index(['unit', 'cycle'], inplace=True)

In [19]:
data

op_set_1  op_set_2  op_set_3  ...  sensor19  sensor20  sensor21
unit cycle                                ...                              
1    1       -0.0007   -0.0004     100.0  ...     100.0     39.06   23.4190
     2        0.0019   -0.0003     100.0  ...     100.0     39.00   23.4236
     3       -0.0043    0.0003     100.0  ...     100.0     38.95   23.3442
     4        0.0007    0.0000     100.0  ...     100.0     38.88   23.3739
     5       -0.0019   -0.0002     100.0  ...     100.0     38.90   23.4044
...              ...       ...       ...  ...       ...       ...       ...
100  196     -0.0004   -0.0003     100.0  ...     100.0     38.49   22.9735
     197     -0.0016   -0.0005     100.0  ...     100.0     38.30   23.1594
     198      0.0004    0.0000     100.0  ...     100.0     38.44   22.9333
     199     -0.0011    0.0003     100.0  ...     100.0     38.29   23.0640
     200     -0.0032   -0.0005     100.0  ...     100.0     38.37   23.0522

[20631 rows x 24 columns]

In [23]:
data.loc[1]

,op_set_1,op_set_2,op_set_3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,sensor7,sensor8,sensor9,sensor10,sensor11,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
cycle,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,-0.0067,0.0003,100.0,518.67,643.75,1602.38,1422.78,14.62,21.61,551.94,2388.31,9037.91,1.3,48.00,519.79,2388.23,8117.69,8.5207,0.03,396,2388,100.0,38.51,22.9588
189,-0.0006,0.0002,100.0,518.67,644.18,1596.17,1428.01,14.62,21.61,550.70,2388.27,9044.55,1.3,48.08,519.58,2388.33,8117.51,8.5183,0.03,395,2388,100.0,38.48,23.1127
190,-0.0027,0.0001,100.0,518.67,643.64,1599.22,1425.95,14.62,21.61,551.29,2388.29,9040.58,1.3,48.33,520.04,2388.35,8112.58,8.5223,0.03,398,2388,100.0,38.49,23.0675


In [6]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

## The Predictron Model Class


In [8]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten
import tensorflow as tf
from tensorflow.keras.losses import MSE

class Predictron(tf.keras.Model):
  def __init__(self, dataset, RUL, max_depth):
    self.inputs = dataset
    self.targets = RUL
    self.learning_rate = 0.001
    self.size = dataset.shape[0]
    self.max_depth = max_depth

    self.rewards = None
    self.gammas = None
    self.lambdas = None
    self.values = None
    self.preturns = None
    self.lambda_preturns = None

    def build(self):
      self.build_model()
      self.build_loss()

    def iter_func(self, state):
      value_net = Flatten(state, 24)
      value_net = BatchNormalization(activation='relu')(value_net)
      value_net = Dense(1)(value_net)

      net = Conv2D(24, (3, 3))(state)
      net = BatchNormalization(activation='relu')(net)
      flat_net = Flatten()(net)

      reward = Dense(24)(flat_net)
      reward = BatchNormalization(activation='relu')(reward)
      reward = Dense(1)(reward)

      gamma = Dense(24)(flat_net)
      gamma = BatchNormalization(activation='relu')(gamma)
      gamma = Dense(1)(gamma)

      lambda_net = Dense(24)(flat_net)
      lambda_net = BatchNormalization(activation='relu')(lambda_net)
      lambda_net = Dense(1)(lambda_net)

      net = Conv2D(24, (3, 3))(net)
      net = BatchNormalization(activation='relu')
      net = Conv2D(24, (3, 3))(net)
      net = BatchNormalization(activation='relu')

      return net, reward, gamma, lambda_net, value_net
    
    def build_model(self):
      state = Conv2D(24, (3, 3))(self.inputs)
      state = BatchNormalization(activation='relu')(state)
      state = Conv2D(24, (3, 3))(state)
      state = BatchNormalization(activation='relu')(state)
      
      iter_template = tf.compat.v1.make_template('iterator', self.iter_func)

      rewards_arr = []
      gammas_arr = []
      lambdas_arr = []
      values_arr = []

      for k in range(self.max_depth):
        state, reward, gamma, lambda_, value = iter_template(state)
        rewards_arr.append(reward)
        gammas_arr.append(gamma)
        lambdas_arr.append(lambda_)
        values_arr.append(value)

      _, _, _, _, value = iter_template(state)
      values_arr.append(value)

      bs = tf.shape(self.inputs)[0]
      self.rewards = tf.stack(rewards_arr, axis=1)
      self.rewards = tf.reshape(self.rewards, [bs, self.max_depth, self.size]) 
      self.rewards = tf.concat(values=[tf.zeros(shape=[bs, 1, self.size], dtype=tf.float32),
                                       self.rewards], axis=1, name='rewards')
      self.gammas = tf.stack(gammas_arr, axis=1)
      self.gammas = tf.reshape(self.gammas, [bs, self.max_depth, self.size])
      self.gammas = tf.concat_v2(values=[tf.ones(shape=[bs, 1, self.size], dtype=tf.float32), self.gammas],
                               axis=1, name='gammas')
      self.lambdas = tf.stack(lambdas_arr, axis=1)
      self.lambdas = tf.reshape(self.lambdas, [-1, self.max_depth, self.size])
      self.values = tf.stack(values_arr, axis=1)
      self.valeus = tf.reshape(self.values, [-1, (self.max_depth +1), self.size])

      self.build_preturns()
      self.build_lambda_preturns()

    def build_preturns(self):
      g_preturns = []
      for k in range(self.max_depth, -1, -1):
        g_k = self.values[:, k, :]
        for kk in range(k, 0, -1):
          g_k = self.rewards[:, kk, :] + self.gammas[:, kk, :] * g_k
        g_preturns.append(g_k)
      g_preturns = g_preturns[::-1]
      self.preturns = tf.stack(g_preturns, axis=1, name='preturns')
      self.g_preturns = tf.reshape(self.g_preturns, [-1, (self.max_depth + 1), self.size])

    def build_lambda_preturns(self):
      g_k = self.values[:, -1, :]
      for k in range(self.max_depth -1, -1, -1):
        g_k = (1 - self.lambdas[:, k, :]) * self.values[:, k, :] + self.lambdas[:, k, :] * (self.rewards[:, k + 1, :] + self.gammas[:, k + 1, :] * g_k)
      self.g_lambda_preturns = g_k

    def build_loss(self):
      self.targets_expanded = tf.expand_dims(self.target, 1)
      self.targets_tiled = tf.tile(self.targets_expanded, [1, self.max_depth +1, 1])
      self.loss_preturns = MSE(self.g_preturns, self.targets_tiled)
      tf.compat.v1.losses.add_loss(self.loss_preturns)
      tf.summary.scalar('Loss Preturns', self.loss_preturns)
      self.loss_lambda_preturns - MSE(self.g_lambda_preturns, self.targets)
      tf.compat.v1.losses.add_loss('Loss Lambda Preturns', self.loss_lambda_preturns)
      self.total_loss = losses.get_total_loss(name='Total Loss')


## The Training Module

In [25]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import six.moves.queue as queue
from six.moves import range
import datetime
import logging
import os
import threading
import time

import numpy as np
import tensorflow as tf


# FLAGS = tf.app.flags.FLAGS

# tf.flags.DEFINE_string('train_dir', './ckpts/predictron_train',
#                        'dir to save checkpoints and TB logs')
# tf.flags.DEFINE_integer('max_steps', 10000000, 'num of batches')
# tf.flags.DEFINE_float('learning_rate', 1e-3, 'learning rate')

# tf.flags.DEFINE_integer('batch_size', 128, 'batch size')
# tf.flags.DEFINE_integer('maze_size', 20, 'size of maze (square)')
# tf.flags.DEFINE_float('maze_density', 0.3, 'Maze density')
# tf.flags.DEFINE_integer('max_depth', 16, 'maximum model depth')
# tf.flags.DEFINE_float('max_grad_norm', 10., 'clip grad norm into this value')
# tf.flags.DEFINE_boolean('log_device_placement', False,
#                         """Whether to log device placement.""")
# tf.flags.DEFINE_integer('num_threads', 10, 'num of threads used to generate mazes.')

train_dir = '/'
max_steps = 10000
learning_rate = 0.0001
batch_size = 32
# define 'size' after dataframe import
max_depth = 4
max_grad_norm = 10.
log_device_placement = False


# logging.basicConfig()
# logger = logging.getLogger('training')
# logger.setLevel(logging.INFO)


def train():
  # config = FLAGS

  # global_step = tf.get_variable(
  #   'global_step', [],
  #   initializer=tf.constant_initializer(0), trainable=False)

  cmapss_df = None
  rul_lables = None

  model = Predictron(cmapss_df, rul_lables, max_depth)
  model.build()

  loss = model.total_loss
  loss_preturns = model.loss_preturns
  loss_lambda_preturns = model.loss_lambda_preturns

  opt = tf.keras.AdamOptimizer(learning_rate=learning_rate)
  grad_vars = opt.get_gradients(loss, tf.trainable_variables())
  grads, vars = zip(*grad_vars)
  grads_clipped, _ = tf.clip_by_global_norm(grads, FLAGS.max_grad_norm)
  grad_vars = zip(grads_clipped, vars)
  apply_gradient_op = opt.apply_gradients(grad_vars, global_step=global_step)

  update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  update_op = tf.group(*update_ops)
  # Group all updates to into a single train op.
  train_op = tf.group(apply_gradient_op, update_op)

  init = tf.global_variables_initializer()
  sess = tf.Session()
  sess.run(init)

  saver = tf.train.Saver(tf.global_variables())
  tf.train.start_queue_runners(sess=sess)

  train_dir = os.path.join(FLAGS.train_dir, 'max_steps_{}'.format(FLAGS.max_depth))
  summary_merged = tf.summary.merge_all()
  summary_writer = tf.summary.FileWriter(train_dir, sess.graph)

  frame_queue = queue.Queue(100)

  def 

  def maze_generator():
    col_names = ['unit', 'cycle', 'op_set_1', 'op_set_2', 'op_set_3',
             'sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6',
             'sensor7', 'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 
             'sensor13', 'sensor14', 'sensor15', 'sensor16', 'sensor17',
             'sensor18', 'sensor19', 'sensor20', 'sensor21']

    data = pd.read_csv('train_FD001.txt',
                      delimiter=' ',
                      names=col_names,
                      index_col=False,
                      skipinitialspace=True
                      )

    data.set_index(['unit', 'cycle'], inplace=True)
    


  for thread_i in range(FLAGS.num_threads):
    t = threading.Thread(target=maze_generator)
    t.start()

  for step in range(FLAGS.max_steps):
    start_time = time.time()
    maze_ims_np, maze_labels_np = frame_queue.get()

    _, loss_value, loss_preturns_val, loss_lambda_preturns_val, summary_str = sess.run(
      [train_op, loss, loss_preturns, loss_lambda_preturns, summary_merged],
      feed_dict={
        maze_ims_ph: maze_ims_np,
        maze_labels_ph: maze_labels_np
      })
    duration = time.time() - start_time

    assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

    if step % 10 == 0:
      num_examples_per_step = FLAGS.batch_size
      examples_per_sec = num_examples_per_step / duration
      sec_per_batch = duration

      format_str = (
        '%s: step %d, loss = %.4f, loss_preturns = %.4f, loss_lambda_preturns = %.4f (%.1f examples/sec; %.3f '
        'sec/batch)')
      logger.info(format_str % (datetime.datetime.now(), step, loss_value, loss_preturns_val, loss_lambda_preturns_val,
                                examples_per_sec, sec_per_batch))

    if step % 100 == 0:
      summary_writer.add_summary(summary_str, step)

    # Save the model checkpoint periodically.
    if step % 1000 == 0 or (step + 1) == FLAGS.max_steps:
      checkpoint_path = os.path.join(train_dir, 'model.ckpt')
      saver.save(sess, checkpoint_path, global_step=step)


def main(argv=None):
  train()
